# 6d reconstruction scan

In [ ]:
import os
from camera import load_camera

#import sys
#sys.path.append("../")

#os.environ["KMP_DUPLICATE_LIB_OK"]="TRUE"

## Setup

In [ ]:
# YAG names as in awa_camera_config.yml
# YAG_A is straight ahead and YAG_B is on bent trayectory
YAG_A_name = "EYG7"
YAG_B_name = "EYG8"

# save image path
save_image_location = "/home/awa/awa_data/00_00_24_EEX_Reconstruction/reconstruction_scan"

## Define the diagnostic

In [ ]:
YAG_A = load_camera(YAG_A_name)

In [ ]:
YAG_A.save_image_location = save_image_location
YAG_A.background_file = save_image_location + "/YAG_A_background.npy"

In [ ]:
YAG_A.measure_background()

In [ ]:
YAG_A.test_measurement()

In [ ]:
YAG_B = load_camera(YAG_B_name)

In [ ]:
YAG_B.save_image_location = save_image_location
YAG_B.background_file = save_image_location + "/YAG_B_background.npy"

In [ ]:
YAG_B.measure_background()

In [ ]:
YAG_B.test_measurement()

In [ ]:
YAG_B.visualize = False
YAG_B.measure_beamsize(20, fit_image=True)

## Define the evaluator

In [ ]:
from epics import caput, caget
from xopt import Evaluator
import time
import numpy as np

def evaluate_function(inputs: dict, diagnostic, testing=False) -> dict:
    # caput values
    if not testing:
        for name, val in inputs.items():
            caput(name, val)
            print(name, val)

    # wait for changes to occur
    time.sleep(2)
    print(time.time())
    results = diagnostic.measure_beamsize(5, fit_image=False, **inputs)

    return results | inputs

evaluator = Evaluator(function=evaluate_function)

## Set up scan

In [ ]:
import numpy as np
import pandas as pd
import yaml
import json

# pv names
quad_pv_name = "AWA:Bira3Ctrl:Ch10"
tdc_pv_name = "pi:QBCR1:Ch7:EnablePulse"
dipole_pv_name = "AWA:Bira4Ctrl:Ch01"

# quad strengths
ks = np.linspace(-0.4, 0.4, 3)

# tdc/dipole values
dipole_on_pv_value = -3.64
tdc_vals = {"OFF":0, "ON":1} # off/on values
dipole_vals = {"OFF":0, "ON":dipole_on_pv_value} # off/on vals

# camera settings
YAG_A.target_charge = 4.5e-9
YAG_B.target_charge = 4.5e-9
YAG_A.charge_atol = 0.5e-9
YAG_B.charge_atol = 0.5e-9
YAG_A.wait_time = 0.6
YAG_B.wait_time = 0.6

In [ ]:
def do_quad_scan(k, pv_dict, diagnostic, testing=False):
    q_scan_data = []
    print("doing quad scan")
    for ele in k:
        # add tcav/dipole settings to quad settings
        total_pvs = pv_dict | {quad_pv_name:ele}
        print(total_pvs)

        # do measurement
        result = evaluate_function(total_pvs, diagnostic, testing)
        q_scan_data += [result]

    return q_scan_data

## Run scan

In [ ]:
data = []

# ----------------------------
# do quad scan with dipole off
# ----------------------------

# set camera
YAG_A.set_camera()

# set dipole
print("setting dipole")
caput(dipole_pv_name, dipole_vals["OFF"])
time.sleep(15)
print("dipole finished moving")

# scan over TDC and quad
for tdc_state in ["OFF","ON"]:
    q_scan_results = do_quad_scan(
        ks,
        {tdc_pv_name: tdc_vals[tdc_state], dipole_pv_name: dipole_vals["OFF"]},
        YAG_A,
    )

    data = data + q_scan_results

    pd_data = pd.DataFrame(data)

    # save dataframe to file
    with open(os.path.join(save_image_location, 'data.yml'), 'w') as outfile:
        yaml.dump(
            json.loads(pd_data.to_json(orient="records")),
            outfile,
            default_flow_style=False
        )

# ----------------------------
# do quad scan with dipole on
# ----------------------------

# set camera
YAG_B.set_camera()

# set dipole
print("setting dipole")
caput(dipole_pv_name, dipole_vals["OFF"])
time.sleep(15)
print("dipole finished moving")

# scan over TDC and quad
for tdc_state in ["OFF","ON"]:
    q_scan_results = do_quad_scan(
        ks,
        {tdc_pv_name: tdc_vals[tdc_state], dipole_pv_name: dipole_vals["ON"]},
        YAG_B,
    )

    data = data + q_scan_results

    pd_data = pd.DataFrame(data)

    # save dataframe to file
    with open(os.path.join(save_image_location, 'data.yml'), 'w') as outfile:
        yaml.dump(
            json.loads(pd_data.to_json(orient="records")),
            outfile,
            default_flow_style=False
        )